In [ ]:
!pip install pandas

In [7]:
import sqlite3
import pandas as pd
import datetime

print("SQLite3 ver",sqlite3.version)

SQLite3 ver 2.6.0


Connect to the Database

In [11]:
con = sqlite3.connect('database_mstr.special_parts_shop_database')
cur = con.cursor()

Create Tables

In [20]:
# Create table
sql = '''
create table Region(
	id INTEGER PRIMARY KEY AUTOINCREMENT,
  	name varchar(25) not null
);
'''
cur.execute(sql)

sql = '''
create table Province(
	id char(2) not null,
	name varchar(10) not null, 
	id_region int not null,
	primary key(id),
	foreign key (id_region) references Region(id)
);
'''
cur.execute(sql)

sql = '''
create table City(
	id INTEGER PRIMARY KEY AUTOINCREMENT,
	name varchar(25) not null, 
	id_province char(2) not null,
	foreign key (id_province) references Province(id)
);
'''
cur.execute(sql)

sql = '''
create table CAP(
	cap_code char(5) not null, 
	id_city int not null,
	primary key (cap_code),
	foreign key (id_city) references City(id)
);
'''
cur.execute(sql)

sql = '''
create table Customer(
	id varchar(36) not null,
	name varchar(30) not null,
	surname varchar(30) not null,
	cap char(5) not null,
	customer_type char(1) not null, 
	fiscal_code varchar(16),
	vat varchar(16),
	primary key (id),
	foreign key (cap) references CAP(cap_code),
	CONSTRAINT `customer_type_chk` CHECK (customer_type in ('C', 'P')),
	CONSTRAINT `is_valid_customer_chk` CHECK (
		(customer_type = 'P' and fiscal_code is not null and vat is null) 
		or (customer_type = 'C' and vat is not null and fiscal_code is null)
	)
);
'''
cur.execute(sql)

sql = '''
create table Payment(
	id char(36) not null, 
	payment_type varchar(10) CHECK( payment_type IN ('card','cash') ) not null,
	total FLOAT not null,
	id_transaction char(10),
	primary key (id),
	CONSTRAINT `is_valid_payment_chk` CHECK (
		(payment_type = 'cash' and id_transaction is null) OR (payment_type = 'card' and id_transaction is not null)
	)
);
'''
cur.execute(sql)

sql = '''
create table `order`(
	id char(36) not null,
	id_customer varchar(36) not null,
	id_payment varchar(36) not null,
	date_order date not null,
	primary key(id),
	foreign key (id_customer) references Customer(id),
	foreign key (id_payment) references Payment(id)
);
'''
cur.execute(sql)

sql = '''
create table Category_Product(
	id INTEGER PRIMARY KEY AUTOINCREMENT,
	name varchar(25) not null
);
'''
cur.execute(sql)

sql = '''
create table Product(
	id INTEGER PRIMARY KEY AUTOINCREMENT,
	name varchar(30) not null,
	description varchar (40) not null, 
	unit_cost float not null,
	selling_price float not null,
	id_category int not null,
	foreign key(id_category) references Category_Product(id)
);
'''
cur.execute(sql)

sql = '''
create table OrderContainsProduct(
	id_product int not null,
	id_order varchar(36) not null, 
	quantity int not null,
	price int not null,
	discount_presence int not null, 
	primary key (id_product, id_order),
	foreign key (id_product) references Product(id),
	foreign key (id_order) references `order`(id)
);
'''
cur.execute(sql)


sql = '''
create table Promotion(
	id INTEGER PRIMARY KEY AUTOINCREMENT,
	percent_off float not null,
	date_start date not null,
	date_finish date not null,
	id_product int not null,
	foreign key (id_product) references Product(id)
);
'''
cur.execute(sql)

sql = '''
create table ProductCompatibleVehicle(
	id_product int not null, 
	id_vehicle int not null,
	primary key(id_product, id_vehicle),
	foreign key(id_product) references Product(id),
	foreign key(id_vehicle) references Vehicle(id)
);
'''
cur.execute(sql)

sql = '''
create table Brand(
	id INTEGER PRIMARY KEY AUTOINCREMENT,
	name char(25) not null
);
'''
cur.execute(sql)

sql = '''
create table Vehicle(
	id INTEGER PRIMARY KEY AUTOINCREMENT,
	name varchar(25) not null,
	type varchar(20) CHECK( type IN ('Suv', 'Supercar', 'Utilitarian', 'Sedan', 'Truck' ) ) not null,
	id_brand int not null,
	foreign key (id_brand) references Brand(id)
);
'''
cur.execute(sql)

# Save (commit) the changes
con.commit()


Insert Data inside Database

In [21]:
sql = '''
INSERT INTO region (name) VALUES
	 ('Sicilia'), ('Puglia'), ('Lombardia'), ('Piemonte'), ('Calabria'), ('Toscana');
'''
cur.execute(sql)

sql = '''
INSERT INTO province (id,name,id_region) VALUES
	 ('BA','Bari',2), ('CT','Catania',1), ('FI','Firenze',6), ('MI','Milano',3), ('TO','Torino',4);
'''
cur.execute(sql)

sql = '''
INSERT INTO city (name,id_province) VALUES
	 ('Catania','CT'), ('Milano','MI'), ('Adrano','CT'), ('Cormano','MI');
'''
cur.execute(sql)

sql = '''
INSERT INTO cap (cap_code,id_city) VALUES
	 ('95121',1), ('95128',1), ('20020',2), ('20021',2), ('95031',3), ('20030',4);
'''
cur.execute(sql)

sql = '''
INSERT INTO customer (id,name,surname,cap,customer_type,fiscal_code,vat) VALUES
	 ('c7d5adf9-d530-11ec-8b01-34e6d73b10fe','John','Light','95031','P','GVNCHR1234567890',NULL),
	 ('c9915286-d530-11ec-8b01-34e6d73b10fe','Mary','Who','95121','C',NULL,'12345678910'),
	 ('cabc4b27-d530-11ec-8b01-34e6d73b10fe','Rose','Light','95128','P','RSELGT1234567890',NULL),
	 ('cba92bfc-d530-11ec-8b01-34e6d73b10fe','Claudio','Michels','20020','P','CLDMCH1234567890',NULL),
	 ('ccd6d70e-d530-11ec-8b01-34e6d73b10fe','Johnny','Michels','20021','P','JHNMCH1234567890',NULL),
	 ('cd9c68f9-d530-11ec-8b01-34e6d73b10fe','Seraphine','Cloud','95121','C',NULL,'12345671234'),
	 ('ce49af91-d530-11ec-8b01-34e6d73b10fe','Joseph','Michels','20020','C',NULL,'12345675678'),
	 ('cf1ab3fe-d530-11ec-8b01-34e6d73b10fe','Andrew','Sunday','20030','P','ANDSND1234567890',NULL),
	 ('cfce0897-d530-11ec-8b01-34e6d73b10fe','Micky ','O'' Connor','95121','C',NULL,'43215678910'),
	 ('d0840a43-d530-11ec-8b01-34e6d73b10fe','Elijah ','Who','20030','C',NULL,'7777678910');
'''
cur.execute(sql)

sql = '''
INSERT INTO payment (id,payment_type,total,id_transaction) VALUES
	 ('3523cc2f-b603-4520-9c0b-518f869687f0','card',35.0,'1111'),
	 ('3c82f3b8-019d-4d35-af20-3193b9b8a7de','card',5.0,'2222'),
	 ('3c94d5bf-97b0-4475-9212-354d924510ca','card',4.0,'3333'),
	 ('82c67a61-9aa1-45c5-aa0c-ec97e6b7c8c9','cash',2.0,NULL),
	 ('99826162-9197-4508-9f67-cd3bd050034c','cash',55.0,NULL),
	 ('f7fdf6bf-3af8-4652-b32a-61890488b3bd','card',55.0,'212321213'),
	 ('fb19c62f-e4e9-448b-ae41-cc340a9814d4','cash',11.0,NULL);
'''
cur.execute(sql)

sql = '''
INSERT INTO `order` (id,id_customer,id_payment,date_order) VALUES
	 ('51e16ad1-d5f1-11ec-8b01-34e6d73b10fe','c7d5adf9-d530-11ec-8b01-34e6d73b10fe','3523cc2f-b603-4520-9c0b-518f869687f0','2022-01-01'),
	 ('51e2ffd7-d5f1-11ec-8b01-34e6d73b10fe','c9915286-d530-11ec-8b01-34e6d73b10fe','3c82f3b8-019d-4d35-af20-3193b9b8a7de','2022-01-02'),
	 ('51e3c0be-d5f1-11ec-8b01-34e6d73b10fe','ccd6d70e-d530-11ec-8b01-34e6d73b10fe','82c67a61-9aa1-45c5-aa0c-ec97e6b7c8c9','2022-01-03'),
	 ('51e46ade-d5f1-11ec-8b01-34e6d73b10fe','ccd6d70e-d530-11ec-8b01-34e6d73b10fe','fb19c62f-e4e9-448b-ae41-cc340a9814d4','2022-01-14'),
	 ('51e50b61-d5f1-11ec-8b01-34e6d73b10fe','cf1ab3fe-d530-11ec-8b01-34e6d73b10fe','99826162-9197-4508-9f67-cd3bd050034c','2022-01-04'),
	 ('ecad129d-a580-4acf-ac31-c8f47771d335','cf1ab3fe-d530-11ec-8b01-34e6d73b10fe','f7fdf6bf-3af8-4652-b32a-61890488b3bd','2022-01-04');
'''
cur.execute(sql)

sql = '''
INSERT INTO category_product (name) VALUES
	 ('Carrozzeria'), ('Elettrico'), ('Vetro'), ('Tergicristalli'), ('Freno');
'''
cur.execute(sql)

sql = '''
INSERT INTO product (name,description,unit_cost,selling_price,id_category) VALUES
	 ('Batteria Avviamento Y3','Applicazione automatica',10.0,35.0,2),
	 ('Specchio retrovisore esterno','Sx, convesso, termico',4.0,11.0,1),
	 ('Specchio retrovisore interno','Sx, convesso, termico',4.0,11.0,1),
	 ('Batteria Avviamento Y5','Applicazione automatica',25.0,55.0,2),
	 ('Lente diffusore ','Faro principale',1.0,5.0,3),
	 ('Lente diffusore crystal','Faro principale',1.0,10.0,3),
	 ('Spazzola Tergi STAR','Spazzola tergi a staffa senza spoiler,',0.6,2.0,4),
	 ('Batteria Avviamento Y7','Applicazione automatica',35.0,75.0,2),
	 ('Spazzola Tergi KODA','Spazzola tergi a staffa senza spoiler,',0.55,2.0,4),
	 ('Pastiglie freno','Assetto posteriore',1.0,4.0,5);
'''
cur.execute(sql)

sql = '''
INSERT INTO ordercontainsproduct (id_product,id_order,quantity,price,discount_presence) VALUES
	 (1,'51e16ad1-d5f1-11ec-8b01-34e6d73b10fe',2,35,0),
	 (1,'51e46ade-d5f1-11ec-8b01-34e6d73b10fe',1,2,1),
	 (2,'51e16ad1-d5f1-11ec-8b01-34e6d73b10fe',1,35,1),
	 (2,'51e2ffd7-d5f1-11ec-8b01-34e6d73b10fe',1,11,1),
	 (2,'51e3c0be-d5f1-11ec-8b01-34e6d73b10fe',4,11,0),
	 (4,'51e46ade-d5f1-11ec-8b01-34e6d73b10fe',10,55,0),
	 (5,'51e2ffd7-d5f1-11ec-8b01-34e6d73b10fe',22,1,0),
	 (5,'51e50b61-d5f1-11ec-8b01-34e6d73b10fe',1,5,0);
'''
cur.execute(sql)

sql = '''
INSERT INTO promotion (percent_off,date_start,date_finish,id_product) VALUES
	 (10.0,'2022-01-01','2022-01-01',2),
	 (10.0,'2022-01-01','2022-01-01',3),
	 (10.0,'2022-01-10','2022-01-15',1),
	 (20.0,'2022-01-30','2022-02-15',4),
	 (20.0,'2022-01-10','2022-01-15',5),
	 (40.0,'2022-01-10','2022-01-15',6),
	 (40.0,'2022-01-01','2022-01-01',7),
	 (50.0,'2022-01-01','2022-01-01',8),
	 (20.0,'2022-01-30','2022-02-15',9),
	 (20.0,'2022-01-30','2022-02-15',10);
'''
cur.execute(sql)

sql = '''
INSERT INTO brand (name) VALUES
	 ('BMW'), ('Mercedes'), ('Volkswagen'), ('Hyundai'), ('Fiat'), ('Alfa Romeo');
'''
cur.execute(sql)

sql = '''
INSERT INTO vehicle (name,`type`,id_brand) VALUES
	 ('Panda','Utilitarian',5), ('500','Utilitarian',5), ('MITO','Utilitarian',6), ('GLA','Suv',2), ('X7','Suv',1);
'''
cur.execute(sql)

sql = '''
INSERT INTO productcompatiblevehicle (id_product,id_vehicle) VALUES
	 (1,1), (1,3), (2,3), (3,3), (4,5), (5,2), (6,4);
'''
cur.execute(sql)

sql = '''
INSERT INTO region (name) VALUES
	 ('Sicilia'),
	 ('Puglia'),
	 ('Lombardia'),
	 ('Piemonte'),
	 ('Calabria'),
	 ('Toscana');
'''
cur.execute(sql)

# Save (commit) the changes
con.commit()

 ### 1. For a specific customer, determine the quantities of each product purchased in a given period

In [22]:
query = '''
select c.id, c.name, p.name, SUM(ocp.quantity) as qty
from `order` o
	inner join customer c on o.id_customer = c.id 
	inner join ordercontainsproduct ocp on ocp.id_order = o.id 
	inner join product p on ocp.id_product = p.id
where c.id = ? and o.date_order between ? and ?
group by p.id
'''

input_customer_id = 'c7d5adf9-d530-11ec-8b01-34e6d73b10fe'
input_start_date = datetime.date(2022, 1, 1)
input_end_date = datetime.date(2022, 1, 20)

cur.execute(query, (input_customer_id, input_start_date, input_end_date ))

pd.DataFrame(cur.fetchall())

,0,1,2,3
0,c7d5adf9-d530-11ec-8b01-34e6d73b10fe,John,Batteria Avviamento Y3,2
1,c7d5adf9-d530-11ec-8b01-34e6d73b10fe,John,Specchio retrovisore esterno,1


 ### 2. For a given product, determine the number of separate customers who purchased it in a given period

In [24]:
query = '''
select count (distinct c.id) as n_customer
from customer as c 
inner join `order` o on c.id = o.id_customer 
inner join ordercontainsproduct o2 on o2.id_order = o.id 
inner join product p on o2.id_product = p.id 
where p.id = ? and o.date_order between ? and ?
'''

input_start_date = datetime.date(2022, 1, 1)
input_end_date = datetime.date(2022, 1, 20)
input_prod_code = 2

cur.execute(query, (input_prod_code, input_start_date, input_end_date ))

pd.DataFrame(cur.fetchall())

,0
0,3


### 3. Identify all customers who have purchased a product in promotion, indicating the cap of residence

In [25]:
query = '''
select c.id, c.name, c.cap
from product p
	inner join promotion promo on p.id = promo.id_product 
	inner join ordercontainsproduct ocp on ocp.id_product = p.id
	inner join `order` o on o.id = ocp.id_order 
	inner join customer c on c.id = o.id_customer 
where ocp.discount_presence = TRUE
group by c.cap 
'''

cur.execute(query)

pd.DataFrame(cur.fetchall())

,0,1,2
0,ccd6d70e-d530-11ec-8b01-34e6d73b10fe,Johnny,20021
1,c7d5adf9-d530-11ec-8b01-34e6d73b10fe,John,95031
2,c9915286-d530-11ec-8b01-34e6d73b10fe,Mary,95121


### 4. For a specific CAP, identify customers who have made purchases in a given period

In [28]:
query = '''
select c.*
from customer as c 
	inner join `order` o on c.id = o.id_customer 
	inner join cap ca on c.cap = ca.cap_code
where ca.cap_code = ? and o.date_order between ? and ?
'''

input_cap = '95031'
input_start_date = datetime.date(2022, 1, 1)
input_end_date = datetime.date(2022, 1, 20)

cur.execute(query, (input_cap, input_start_date, input_end_date ))

pd.DataFrame(cur.fetchall())

,0,1,2,3,4,5,6
0,c7d5adf9-d530-11ec-8b01-34e6d73b10fe,John,Light,95031,P,GVNCHR1234567890,None


### 5. For a specific car model, identify the revenue of the items in a given period

In [30]:
query = '''
select SUM(ocp.price * ocp.quantity) as total_with_discount, v.name, op.date_order, pr.id, pr.name
from vehicle v 
	inner join productcompatiblevehicle pcv on v.id = pcv.id_vehicle 
	inner join product pr on pr.id = pcv.id_product
	inner join ordercontainsproduct ocp on ocp.id_product = pr.id 
	inner join `order` op on op.id = ocp.id_order 
	inner join payment pay on pay.id = op.id_payment 
where v.id = ? and op.date_order between ? and ?
group by pr.id
'''

input_vehicle_id = 3
input_start_date = datetime.date(2022, 1, 1)
input_end_date = datetime.date(2022, 1, 20)

cur.execute(query, (input_vehicle_id, input_start_date, input_end_date ))

pd.DataFrame(cur.fetchall())

,0,1,2,3,4
0,72,MITO,2022-01-01,1,Batteria Avviamento Y3
1,90,MITO,2022-01-01,2,Specchio retrovisore esterno


### 6. For each item in promo determine the quantity sold in a given period 

In [31]:
query = '''
select pro.id, pro.name, SUM(ocp.quantity) 
from product pro 
	inner join ordercontainsproduct ocp on pro.id = ocp.id_product 
	inner join `order` o on o.id = ocp.id_order 
	inner join payment pay on o.id_payment = pay.id 
where o.date_order between ? and ? and ocp.discount_presence = TRUE
group by pro.id
'''

input_start_date = datetime.date(2022, 1, 1)
input_end_date = datetime.date(2022, 1, 20)

cur.execute(query, (input_start_date, input_end_date ))

pd.DataFrame(cur.fetchall())

,0,1,2
0,1,Batteria Avviamento Y3,1
1,2,Specchio retrovisore esterno,2


### 7. Identify the customers who have not made an electronic payment in the current year

In [32]:
query = '''
select distinct c.*
from customer c
	inner join `order` o on c.id = o.id_customer 
	inner join ordercontainsproduct ocp on ocp.id_order = o.id 
	inner join payment pay on o.id_payment = pay.id 
where pay.payment_type = "cash" and c.id not in (
	select c2.id 
	from customer c2 
		inner join `order` o2 on o2.id_customer = c.id
		inner join payment p2 on o2.id_payment = p2.id 
	where p2.payment_type = 'card'
)
'''

cur.execute(query, ())

pd.DataFrame(cur.fetchall())

,0,1,2,3,4,5,6
0,ccd6d70e-d530-11ec-8b01-34e6d73b10fe,Johnny,Michels,20021,P,JHNMCH1234567890,None


### 8. Identify the customer who spent more in a given period

In [33]:
query = '''
select c.id, MAX(p.total) totale, prod.name, ocp.quantity, o.date_order 
from customer c 
	inner join `order` o on c.id = o.id_customer 
	inner join ordercontainsproduct ocp on ocp.id_order = o.id 
	inner join product prod on ocp.id_product = prod.id 
	inner join payment p on p.id = o.id_payment 
where o.date_order between ? and ?
group by c.id
order by totale desc
limit 1
'''

input_start_date = datetime.date(2022, 1, 1)
input_end_date = datetime.date(2022, 1, 20)

cur.execute(query, (input_start_date, input_end_date ))

pd.DataFrame(cur.fetchall())

,0,1,2,3,4
0,cf1ab3fe-d530-11ec-8b01-34e6d73b10fe,55.0,Lente diffusore,1,2022-01-04


### 9. For a specific CAP the average expense per invoice in a given period

In [36]:
query = '''
select c.cap, AVG(pay.total) as spesa_media
from payment pay 
	inner join `order` o on o.id_payment = pay.id 
	inner join customer c on c.id = o.id_customer
where c.cap = ? and o.date_order between ? and ?
'''

input_cap = '20021'
input_start_date = datetime.date(2022, 1, 1)
input_end_date = datetime.date(2022, 1, 14)

cur.execute(query, (input_cap, input_start_date, input_end_date ))

pd.DataFrame(cur.fetchall())

,0,1
0,20021,6.5


### 10. For each category of products the total price paid for those sold in a given period.



In [40]:
query = '''
select cp.id, cp.name, SUM(ocp.price*ocp.quantity)  
from product p 
	inner join ordercontainsproduct ocp on p.id = ocp.id_product 
	inner join category_product cp on p.id_category = cp.id 
	inner join `order` o on ocp.id_order = o.id 
	inner join payment pay on o.id_payment = pay.id 
where o.date_order between ? and ?
group by cp.id 
order by pay.total desc
'''

input_start_date = datetime.date(2022, 1, 1)
input_end_date = datetime.date(2022, 2, 15)

cur.execute(query, (input_start_date, input_end_date ))

pd.DataFrame(cur.fetchall())

,0,1,2
0,2,Elettrico,622
1,1,Carrozzeria,90
2,3,Vetro,27


Close the connection

In [15]:
cur.close()
con.close()